In [4]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split
%matplotlib inline

In [5]:
dataframe = pd.read_csv('D:\PyTorch\covid19India.csv')

In [6]:
dataframe.head()
df = dataframe.loc[(dataframe['State/UnionTerritory'] == 'Maharashtra')]
df

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
76,77,09/03/20,6:00 PM,Maharashtra,2,0,0,0,2
91,92,10/03/20,6:00 PM,Maharashtra,5,0,0,0,5
97,98,11/03/20,6:00 PM,Maharashtra,2,0,0,0,2
120,121,12/03/20,6:00 PM,Maharashtra,11,0,0,0,11
133,134,13/03/20,6:00 PM,Maharashtra,14,0,0,0,14
...,...,...,...,...,...,...,...,...,...
2361,2362,27/05/20,8:00 AM,Maharashtra,-,-,16954,1792,54758
2397,2398,28/05/20,8:00 AM,Maharashtra,-,-,17918,1897,56948
2433,2434,29/05/20,8:00 AM,Maharashtra,-,-,17918,1897,56948
2469,2470,30/05/20,8:00 AM,Maharashtra,-,-,26997,2098,62228


In [7]:
dataframe.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30/01/20,6:00 PM,Kerala,1,0,0,0,1
1,2,31/01/20,6:00 PM,Kerala,1,0,0,0,1
2,3,01/02/20,6:00 PM,Kerala,2,0,0,0,2
3,4,02/02/20,6:00 PM,Kerala,3,0,0,0,3
4,5,03/02/20,6:00 PM,Kerala,3,0,0,0,3


In [8]:
dataframe.shape[0]

2522

In [9]:
dataframe.shape[1]

9

In [10]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522 entries, 0 to 2521
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       2522 non-null   int64 
 1   Date                      2522 non-null   object
 2   Time                      2522 non-null   object
 3   State/UnionTerritory      2522 non-null   object
 4   ConfirmedIndianNational   2522 non-null   object
 5   ConfirmedForeignNational  2522 non-null   object
 6   Cured                     2522 non-null   int64 
 7   Deaths                    2522 non-null   int64 
 8   Confirmed                 2522 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 177.5+ KB


In [11]:
dataframe.isnull().sum()

Sno                         0
Date                        0
Time                        0
State/UnionTerritory        0
ConfirmedIndianNational     0
ConfirmedForeignNational    0
Cured                       0
Deaths                      0
Confirmed                   0
dtype: int64

In [12]:
#Obtaining the relevant columns
#df = dataframe.iloc[:, [0,1,2,3,4,5,6]]
#df = dataframe.drop(['deltaconfirmed', 'deltadeaths', 'deltarecovered', 'statenotes', 'lastupdatedtime'], axis = 1)
df = dataframe.drop(['Date', 'Time'], axis = 1)
df = df.loc[(dataframe['State/UnionTerritory'] == 'Maharashtra')]
df

,Sno,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
76,77,Maharashtra,2,0,0,0,2
91,92,Maharashtra,5,0,0,0,5
97,98,Maharashtra,2,0,0,0,2
120,121,Maharashtra,11,0,0,0,11
133,134,Maharashtra,14,0,0,0,14
...,...,...,...,...,...,...,...
2361,2362,Maharashtra,-,-,16954,1792,54758
2397,2398,Maharashtra,-,-,17918,1897,56948
2433,2434,Maharashtra,-,-,17918,1897,56948
2469,2470,Maharashtra,-,-,26997,2098,62228


In [13]:
df.head()

,Sno,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
76,77,Maharashtra,2,0,0,0,2
91,92,Maharashtra,5,0,0,0,5
97,98,Maharashtra,2,0,0,0,2
120,121,Maharashtra,11,0,0,0,11
133,134,Maharashtra,14,0,0,0,14


In [14]:
df._get_numeric_data().columns

Index(['Sno', 'Cured', 'Deaths', 'Confirmed'], dtype='object')

In [15]:
num_rows = df.shape[0]
num_cols = df.shape[1]
input_cols = df.columns.values.tolist()#['confirmed', 'active', 'deaths']
output_cols = [df.columns[-1]]
categorical_cols = list(set(df.columns) - set(df._get_numeric_data().columns))
print(categorical_cols)
print(num_rows, num_cols)
print(input_cols, output_cols)
df.info()

['ConfirmedForeignNational', 'ConfirmedIndianNational', 'State/UnionTerritory']
84 7
['Sno', 'State/UnionTerritory', 'ConfirmedIndianNational', 'ConfirmedForeignNational', 'Cured', 'Deaths', 'Confirmed'] ['Confirmed']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 76 to 2505
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       84 non-null     int64 
 1   State/UnionTerritory      84 non-null     object
 2   ConfirmedIndianNational   84 non-null     object
 3   ConfirmedForeignNational  84 non-null     object
 4   Cured                     84 non-null     int64 
 5   Deaths                    84 non-null     int64 
 6   Confirmed                 84 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 5.2+ KB


In [16]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

In [17]:
inputs_array, targets_array = dataframe_to_arrays(df)
inputs_array, targets_array

(array([[   77,     0,     7,     1,     0,     0,     2],
        [   92,     0,    13,     1,     0,     0,     5],
        [   98,     0,     7,     1,     0,     0,     2],
        [  121,     0,     1,     1,     0,     0,    11],
        [  134,     0,     5,     1,     0,     0,    14],
        [  147,     0,     5,     1,     0,     0,    14],
        [  154,     0,     8,     1,     0,     0,    32],
        [  168,     0,     8,     1,     0,     0,    32],
        [  183,     0,     9,     2,     0,     1,    39],
        [  198,     0,    10,     2,     0,     1,    42],
        [  216,     0,    11,     2,     0,     1,    47],
        [  236,     0,    12,     2,     0,     1,    52],
        [  258,     0,    14,     2,     0,     1,    63],
        [  281,     0,    15,     2,     0,     2,    67],
        [  304,     0,    16,     2,     0,     2,    74],
        [  327,     0,    17,     2,     0,     2,    89],
        [  351,     0,     3,     2,     1,     3,   128

In [18]:
inputs = torch.from_numpy(inputs_array).float()
targets = torch.from_numpy(targets_array).float()

In [19]:
dataset = TensorDataset(inputs, targets)

In [20]:
# Splitting
val_percent = 0.1
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size

train_ds, val_ds = random_split(dataset, [train_size,val_size])

In [21]:
print(len(train_ds), len(val_ds))

76 8


In [22]:
batch_size = 64

In [23]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [24]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[2.8100e+02, 0.0000e+00, 1.5000e+01, 2.0000e+00, 0.0000e+00, 2.0000e+00,
         6.7000e+01],
        [7.2700e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.6000e+01, 4.8000e+01,
         8.6800e+02],
        [2.2910e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.4600e+04, 1.6350e+03,
         5.0231e+04],
        [8.2000e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.2500e+02, 9.7000e+01,
         1.3640e+03],
        [1.4970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.7730e+03, 4.5900e+02,
         1.0498e+04],
        [1.8230e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.1990e+03, 8.3200e+02,
         2.2171e+04],
        [2.1880e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1726e+04, 1.4540e+03,
         4.1642e+04],
        [1.9880e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.5640e+03, 1.0680e+03,
         2.9100e+04],
        [6.0600e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.2000e+01, 1.6000e+01,
         3.3500e+02],
        [2.1540e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0318e+

In [25]:
input_size = len(input_cols)
output_size = len(output_cols)

In [26]:
class FFModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, xb):
        out = self.linear(xb)
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 10th epoch
        if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [27]:
ffmodel = FFModel()

In [28]:
list(ffmodel.parameters())

[Parameter containing:
 tensor([[ 0.0571, -0.2443, -0.1706,  0.2239, -0.2153, -0.3284, -0.0985]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0871], requires_grad=True)]

In [29]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [30]:
result = evaluate(ffmodel, val_loader)
print(result)

{'val_loss': 25275.615234375}


In [31]:
epochs = 100
lr = 5e-7
history5 = fit(epochs, lr, ffmodel, train_loader, val_loader)

Epoch [10], val_loss: 22277.3398
Epoch [20], val_loss: 19303.9023
Epoch [30], val_loss: 16434.5859
Epoch [40], val_loss: 13605.2510
Epoch [50], val_loss: 10704.7939
Epoch [60], val_loss: 7941.3149
Epoch [70], val_loss: 5078.4692
Epoch [80], val_loss: 2145.2573
Epoch [90], val_loss: 121.8287
Epoch [100], val_loss: 241.7931


In [33]:
epochs = 200
lr = 4e-7
history5 = fit(epochs, lr, ffmodel, train_loader, val_loader)

Epoch [10], val_loss: 118.3130
Epoch [20], val_loss: 30.2840
Epoch [30], val_loss: 96.9579
Epoch [40], val_loss: 78.7054
Epoch [50], val_loss: 35.5260
Epoch [60], val_loss: 96.8749
Epoch [70], val_loss: 39.9174
Epoch [80], val_loss: 111.4790
Epoch [90], val_loss: 65.6026
Epoch [100], val_loss: 85.6043
Epoch [110], val_loss: 40.5315
Epoch [120], val_loss: 47.8722
Epoch [130], val_loss: 34.9569
Epoch [140], val_loss: 116.8517
Epoch [150], val_loss: 41.1602
Epoch [160], val_loss: 73.6177
Epoch [170], val_loss: 143.2049
Epoch [180], val_loss: 77.6125
Epoch [190], val_loss: 76.5864
Epoch [200], val_loss: 30.4486


In [34]:
val_loss = 30.4486

In [35]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [36]:
input, target = val_ds[0]
predict_single(input, target, ffmodel)

Input: tensor([ 978.,    0.,    0.,    0.,  259.,  178., 2687.])
Target: tensor([2687.])
Prediction: tensor([2679.2427])


In [37]:
input, target = val_ds[3]
predict_single(input, target, ffmodel)

Input: tensor([ 2470.,     0.,     0.,     0., 26997.,  2098., 62228.])
Target: tensor([62228.])
Prediction: tensor([62251.6445])


In [38]:
input, target = val_ds[5]
predict_single(input, target, ffmodel)

Input: tensor([ 1625.,     0.,     0.,     0.,  2465.,   583., 14541.])
Target: tensor([14541.])
Prediction: tensor([14569.7842])


In [44]:
input, target = val_ds[2]
predict_single(input, target, ffmodel)

Input: tensor([134.,   0.,   5.,   1.,   0.,   0.,  14.])
Target: tensor([14.])
Prediction: tensor([14.4657])
